In [ ]:
#include <iostream>
using namespace std;

typedef long long ll;
typedef long double ld;

cout << "Loaded!" << endl;

In [ ]:
#include <cmath>
#include <iostream>
#include <vector>

#include "../include/plotpp/plotpp.hpp"


using namespace std;


// Generate a damped sine signal and display it inline with plotpp.
vector<double> x(400), y(400);
for (size_t i = 0; i < x.size(); ++i) {
    const double t = static_cast<double>(i) / (x.size() - 1) * 8.0;
    x[i] = t;
    y[i] = sin(6.28318530718 * t) * exp(-0.35 * t);
}


plotpp::cla();
plotpp::figure_size(900, 450);
plotpp::title("Damped sine via plotpp");
plotpp::xlabel("t");
plotpp::ylabel("f(t)");
plotpp::legend(true);
plotpp::named_plot("damped", x, y, "with lines lw 2 lc rgb '#1f77b4'");


const int rc = plotpp::show();
if (rc != 0) {
    cerr << "gnuplot exited with code " << rc << endl;
}

In [ ]:
#include <iostream>
#include <string>
#include <utility>
#include <vector>

#pragma cling add_include_path("../vendor/DataFrame-2.0.0/include")

#include <DataFrame/DataFrame.h>
#include <DataFrame/DataFrameStatsVisitors.h>

using namespace hmdf;
using SensorFrame = StdDataFrame<unsigned long>;

SensorFrame df;
std::vector<unsigned long> idx {0, 1, 2, 3, 4, 5, 6};
std::vector<double> temp_c {21.8, 22.4, 23.1, 24.9, 25.7, 26.4, 27.2};
std::vector<double> humidity_pct {42.0, 41.4, 40.1, 38.9, 37.5, 36.1, 35.3};

df.load_data(std::move(idx),
                std::make_pair("temperature_c", temp_c),
                std::make_pair("humidity_pct", humidity_pct));

MeanVisitor<double, unsigned long> temp_mean;
StdVisitor<double, unsigned long> temp_std;

df.visit<double>("temperature_c", temp_mean);
df.visit<double>("temperature_c", temp_std);

std::cout << "Avg temp: " << temp_mean.get_result()
            << ", stddev: " << temp_std.get_result() << std::endl;

// In DataFrame-2.0.0, the selection functor is called with (index, value).
const auto hot_selector = [](const unsigned long &, const double &value) -> bool {
    return value >= 25.0;
};

auto hot_df = df.get_data_by_sel<double, decltype(hot_selector), double, double>(
                "temperature_c", hot_selector);

const auto &hot_idx = hot_df.get_index();
const auto &hot_temps = hot_df.get_column<double>("temperature_c");
const auto &hot_humidity = hot_df.get_column<double>("humidity_pct");

std::cout << "Hot readings found: " << hot_idx.size() << std::endl;
for (std::size_t i = 0; i < hot_idx.size(); ++i) {
    std::cout << "idx=" << hot_idx[i]
                << ", temp=" << hot_temps[i]
                << ", humidity=" << hot_humidity[i] << std::endl;
}

In [ ]:
#include <iostream>
#include <string>
#include <vector>

#include "../include/plotpp/plotpp.hpp"

#pragma cling add_include_path("../vendor/matplotplusplus/source")
#pragma cling add_library_path("../vendor/matplotplusplus/build-xcpp17/source/matplot")
#pragma cling load("../vendor/matplotplusplus/build-xcpp17/source/matplot/libmatplot.so.1.2.0")

#include <matplot/matplot.h>

try {
    using namespace matplot;

    std::vector<double> y = {29, 17, 14, 13, 12, 4, 11};
    bar(y);
    title("Matplot++ bar chart (inline SVG, no filename)");

    // No manual filename: plotpp creates a temp SVG, displays it, then deletes it.
    plotpp::display_svg_temp([&](const std::string& p) { save(p); });
} catch (const std::exception &e) {
    std::cerr << "Matplot++ error: " << e.what() << std::endl;
    std::cerr << "Tip: Matplot++ needs gnuplot available at runtime." << std::endl;
}